In [ ]:
import numpy as np
import scipy as sp
import time
import matplotlib.pyplot as plt

# Sparse Matrix Formats

Scipy provides several standard types of sparse matrices in `scipy.sparse`.  See the [documentation](https://docs.scipy.org/doc/scipy/reference/sparse.html#sparse-matrix-classes).


## COOrdinate Format

In [ ]:
row = [ 0,  0,  1,  1,  2,  2,  2,  3]
col = [ 0,  1,  1,  3,  2,  3,  4,  5]
val = [10, 20, 30, 40, 50, 60, 70, 80]

S = sp.sparse.coo_matrix((val, (row,col)), shape=(4,6))
print(S)

Using `toarray` command, one can convert sparse matrix format to a dense array.

In [ ]:
S.toarray()

One can visualize the sparsity pattern using PyPlot's `spy` function (this is particularly useful for large sparse matrices)

In [ ]:
plt.spy(S)
plt.show()

## Compressed Sparse Row Formats

In [ ]:
val = [10, 20, 30, 40, 50, 60, 70, 80]
col = [ 0,  1,  1,  3,  2,  3,  4,  5]
row_ptr = [ 0,  2,  4,  7,  8]

S = sp.sparse.csr_matrix((val, col, row_ptr), shape=(4,6))
print(S)

In [ ]:
S.toarray()

In [ ]:
S

## Changing Formats

To convert between sparse matrix formats, you can use `tocsr`, `tocoo`, etc.

In [ ]:
A = S.tocoo()
A

In [ ]:
B = A.tocsr()
B

## Saving and Loading Sparse Matrices

In [ ]:
data = np.loadtxt('1138_bus.mtx', comments='%') # skip any rows that begin with `%`
data.shape

The first non-comment row contains the size of the matrix, so we can handle it separately.

In [ ]:
m, n = int(data[0,0]), int(data[0,1])
data = data[1:]
print(m,n)

Note that indices in matrix market format `.mtx` begin at 1.

In [ ]:
rows = data[:,0] - 1
cols = data[:,1] - 1
vals = data[:,2]
A = sp.sparse.coo_matrix((vals, (rows, cols)), shape=(m,n))

plt.spy(A)
plt.show()

## Sparse vs Dense

Let us look at the difference between using the sparse matrix and a dense matrix for matrix-vector multiplications.

In [ ]:
n = 100
x = np.random.randn(n)
A = sp.sparse.random(n, n, 0.01) + sp.sparse.eye(n)
A

In [ ]:
plt.spy(A)
plt.show()

In [ ]:
y = np.empty_like(x)

t_start = time.time()
y = A @ x
t_end = time.time()
tcsr = t_end - t_start
print("time for CSR multiply: %f sec" % tcsr)

In [ ]:
Adense = A.todense()

t_start = time.time()
y = Adense @ x
t_end = time.time()
tdense = t_end - t_start
print("time for dense multiply: %f sec\n" % tdense)

print("CSR is %f times faster" % (tdense / tcsr))

## Sparse Linear Algebra

Routines for sparse linear algebra are found in `scipy.sparse.linalg`. In particular, function [`scipy.linalg.splu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html#scipy.sparse.linalg.splu) (SParse LU) an LU factorization to solve linear sparse systems.

In [ ]:
A = A.tocsc() # need to convert to CSC form first
LU = sp.sparse.linalg.splu(A)
LU

In [ ]:
y = A @ x

x2 = LU.solve(y)
print(np.linalg.norm(x2 - x))

One can also use `scipy.sparse.linalg.spsolve` function, which wraps this factorization.

In [ ]:
t_start = time.time()
x2 = sp.sparse.linalg.spsolve(A, y)
t_end = time.time()

print("elapsed time:", t_end - t_start, "sec")
print(np.linalg.norm(x2 - x))

# Iterative Methods

Generate a large sparse random matrix $A$.

In [ ]:
n = 1000
A = sp.sparse.random(n, n, density=0.0001, format='csr') + sp.sparse.eye(n, format='csr') * n

plt.spy(A)
plt.show()

In [ ]:
x = np.random.rand(n)
y = A @ x

## Jacobi Iteration

## Dense Matrix

In [ ]:
def solve_jacobi(A, y, x0, TOL=1.0e-7, maxIter=1000):
  n = len(y)
  Dinv = np.diag(1.0 / np.diag(A))
  for k in range(maxIter):
    ALU = np.tril(A,-1) + np.triu(A,1)
    x = y - (ALU @ x0)
    x = Dinv @ x

    if np.linalg.norm(x-x0)<TOL:
      break
    x0 = x

  if k == maxIter-1:
    print('\nMaximum number of iterations reached!')

  return x

In [ ]:
Adense = A.todense()
x0 = np.ones(n)   # initial guess

t_start = time.time()
x1 = solve_jacobi(Adense, y, x0)
t_end = time.time()

print("elapsed time:", t_end - t_start, "sec")
print(np.linalg.norm(x1 - x))

## Sparse Matrix

In [ ]:
def solve_jacobi_csr(A, y, x0, TOL=1.0e-7, maxIter=1000):
  n = len(y)
  Dinv = sp.sparse.diags(1.0 / A.diagonal(), 0, format='csr')
  for k in range(maxIter):
    ALU = sp.sparse.tril(A,-1,'csr') + sp.sparse.triu(A,1,'csr')
    x = y - (ALU @ x0)
    x = Dinv @ x

    if np.linalg.norm(x-x0)<TOL:
      break
    x0 = x

  if k > maxIter:
    print('\nMaximum number of iterations reached!', np.linalg.norm(x-x0))

  return x

In [ ]:
t_start = time.time()
x2 = solve_jacobi_csr(A, y, x0)
t_end = time.time()

print("elapsed time:", t_end - t_start, "sec")
print(np.linalg.norm(x2 - x))

## Tridiagonal System

In [ ]:
n = 200

a = np.full(n, 1)
b = np.full(n, 2)
A = sp.sparse.spdiags([a,b,a], [-1,0,1],n,n,format='csr') + sp.sparse.random(n, n, density=0.001, format='csr')
plt.spy(A)
plt.show()

In [ ]:
u0 = np.zeros(n)
y = np.random.random(n)

u = solve_jacobi_csr(A, y, u0)
plt.plot(u)
plt.show()

# Try this!

Write a code that implements Gauss-Seidel Method. Generate a "noisy" tridiagonal matrix $A$ of your choice, and a random vector $y$. Use your code, to solve linear system $Ax = y$, and plot your solution.
